In [8]:
import os
import dataflows as DF
from datapackage_pipelines_migdar.flows.dump_to_es import my_dump_to_es, BoostingMappingGenerator

# kubectl port-forward to the migdar elasticsearch
os.environ["DATAFLOWS_ELASTICSEARCH"] = "localhost:9200"

# download from the pipelines pod data/ folder
data_source = '../data/publications_in_es/datapackage.json'

revision = 202538
resource_name = 'publications'

DF.Flow(
    DF.load(data_source),
    my_dump_to_es(
        indexes={
            'migdar__' + resource_name: [
                {
                    'resource-name': resource_name,
                    'revision': revision
                }
            ]
        },
        mapper_cls=BoostingMappingGenerator,
        index_settings={'index.mapping.coerce': True},
        elasticsearch_options=dict(timeout=60)
    ),
).process()

(<datapackage.package.Package at 0x77a0152f3f90>,
 {'count_of_rows': 4363,
  'bytes': None,
  'hash': '5f5818e16db4ba82b5038b9c42c4ea61',
  'dataset_name': None})